In [1]:
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.3. Cytokine Distribution Analyses/1.3.1. Cytokine Distribution Comparison/Input')
Output = ('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/1. Compartment Analysis/1.3. Cytokine Distribution Analyses/1.3.1. Cytokine Distribution Comparison/Output')
cur_date = "052621"

library(readxl)
library(tidyverse)
library(reshape2)

#reading in file
data_df <- data.frame(read_excel("CytokineData_102920.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
head(data_df)

,Original_Identifier,Protein,Conc,Conc_pslog2,Compartment,SubjectNo,Group,SubjectID
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,E_C_F_002,IFNg,17.642316,4.220509,NELF,1,NS,NS_1
2,E_C_F_002,IL10,2.873724,1.953721,NELF,1,NS,NS_1
3,E_C_F_002,IL12p70,1.625272,1.392467,NELF,1,NS,NS_1
4,E_C_F_002,IL13,36.117692,5.214035,NELF,1,NS,NS_1
5,E_C_F_002,IL1b,104.409217,6.719857,NELF,1,NS,NS_1
6,E_C_F_002,IL6,21.159536,4.469856,NELF,1,NS,NS_1


A Wilcoxon Rank Sum tests for a difference in distributions b/w two samples (in this case cytokines). First seeing if there are differences in compartments at baseline followed by overall differences based on smoking status. Final analysis included in manuscript includes wilcoxon test across smoking status stratified by compartment. 

In [3]:
#creating vectors to loop through
cytokines = sort(unique(data_df$Protein)[!is.na(unique(data_df$Protein))]) #alphabetizing and removing NAs
compartment = c('NLF','NELF','Sputum','Serum')
smokers = c('CS', 'Ecig')


values = data.frame()
wilcoxon_rank_sum_values = function(dataframe, empty_df){
    #"""
    #Running wilcoxon rank sum tests after filtering for compartment, protein, and smoking status using a loop. 
    #Ultimately using this test to compare subjects (NS vs. Ecig or CS) within the same demographic.
    
    #:param: subsetted dataframe, empty dataframe
    #:output: a dataframe containing the compartment, comparison, protein, u stat, p value

    #"""
    for (i in 1:length(compartment)){
        for (j in 1:length(smokers)){
            for(k in 1:length(cytokines)){
                NS_df = dataframe %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == 'NS') %>%
                    select(Compartment, Protein, Conc_pslog2)
                smokers_df = dataframe %>%
                    filter(Compartment == compartment[i], Protein == cytokines[k], Group == smokers[j]) %>%
                    select(Compartment, Protein, Conc_pslog2)

                #running wilcoxon rank sum
                wilcox_test = wilcox.test(NS_df$Conc_pslog2, smokers_df$Conc_pslog2)
                
                #calculating FC to get directionality
                FC = mean(smokers_df$Conc_pslog2)/mean(NS_df$Conc_pslog2)

                #contains smoking status compared, compartment, cytokine, u stat, and p value
                values_vector = cbind(compartment[i], smokers[j], cytokines[k], wilcox_test$statistic, FC, wilcox_test$p.value)
                empty_df = rbind(empty_df, values_vector)

            }
        }
    }
    colnames(empty_df) = c("Compartment",'Group', 'Cyotkine', 'Stat', 'AD','P Value')
    
    #adding a column for padj
    PAdj = c()
    for (i in 1:length(compartment)){
        single_compartment_df = empty_df %>%
            filter(Compartment == compartment[i])
        padj =  p.adjust(as.numeric(as.character(single_compartment_df$`P Value`)), method = "fdr") 
        PAdj = c(PAdj, padj)
        
    }
    
    empty_df = cbind(empty_df, PAdj)
    return(empty_df)
}

#calling fn
wilcoxon_df = wilcoxon_rank_sum_values(data_df, values)

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(NS_df$Conc_pslog2, smokers_df$Conc_pslog2):
“cannot compute exact p-value with ties”
Warning 

In [4]:
#viewing results
head(wilcoxon_df)

,Compartment,Group,Cyotkine,Stat,AD,P Value,PAdj
,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>
W,NLF,CS,Eotaxin,79.5,1.06735668276928,0.593432010058319,0.9831256
W1,NLF,CS,Eotaxin3,84.5,1.02155028481818,0.770897292144351,0.9831256
W2,NLF,CS,Fractalkine,107,1.01661039638015,0.45826874660365,0.9831256
W3,NLF,CS,I309,91,NaN,NaN,NaN
W4,NLF,CS,IFNg,96.5,0.858408025461366,0.808003880176672,0.9831256
W5,NLF,CS,IL10,86,1.36835048589195,0.829983797231071,0.9831256


In [5]:
#exporting tables
write.csv(wilcoxon_df, paste0(Output,"/", cur_date, "_Cytokine_Wilcoxon_Results.csv"), row.names = FALSE)